In [ ]:
#!pip install --upgrade tensorflow==1.15

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNNCell, RNN, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas_datareader as pdr

In [ ]:
df = pdr.get_data_yahoo('BTC-USD', start = '2019-01-01', end='2021-04-12')
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-01-01,3850.913818,3707.231201,3746.713379,3843.520020,4.324201e+09,3843.520020
2019-01-02,3947.981201,3817.409424,3849.216309,3943.409424,5.244857e+09,3943.409424
2019-01-03,3935.685059,3826.222900,3931.048584,3836.741211,4.530215e+09,3836.741211
2019-01-04,3865.934570,3783.853760,3832.040039,3857.717529,4.847965e+09,3857.717529
2019-01-05,3904.903076,3836.900146,3851.973877,3845.194580,5.137610e+09,3845.194580
...,...,...,...,...,...,...
2021-04-07,58338.738281,55879.085938,56099.914062,58323.953125,5.305386e+10,58323.953125
2021-04-08,58937.046875,57807.863281,58326.562500,58245.003906,4.665521e+10,58245.003906
2021-04-09,61276.664062,58038.707031,58253.777344,59793.234375,5.823847e+10,59793.234375


In [ ]:
prices = df['Close'].values

In [ ]:
train_prices, test_prices = train_test_split(prices, test_size = 0.2, shuffle = False)



In [ ]:
#reshape jadi column matrix
train_prices = train_prices.reshape(-1, 1)
test_prices = test_prices.reshape(-1,1)

In [ ]:
#preprocess
scaler = MinMaxScaler().fit(train_prices)
train_prices = scaler.transform(train_prices)
test_prices = scaler.transform(test_prices)

In [ ]:
timestep = 64

In [ ]:
def construct_dataset(in_prices):
    features = []
    targets = []
    for i in range(timestep, in_prices.shape[0]):
        features.append(
            in_prices[i - timestep : i]
        )
        targets.append(
            in_prices[i]
        )

    features = np.array(features, 'float32')
    targets = np.array(targets, 'float32')

    return features, targets

x_train, y_train = construct_dataset(train_prices)
x_test, y_test = construct_dataset(test_prices)

In [ ]:
input_placeholder = tf.placeholder(tf.float32, [None, timestep, 1])
target_placeholder = tf.placeholder(tf.float32, [None, 1])

In [ ]:
cell = SimpleRNNCell(64, 'relu')
rnn_layer = RNN(cell, dtype = tf.float32)
output_layer = Dense(1)

In [ ]:
out_tensor = rnn_layer(input_placeholder)
out_tensor = output_layer(out_tensor)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Training

In [ ]:
#MSE
loss_tensor = tf.reduce_mean(0.5 * (out_tensor - target_placeholder) ** 2)

#Optional

rmse_tensor = tf.reduce_mean(tf.sqrt(0.5 * (out_tensor - target_placeholder) ** 2))


In [ ]:
optimizer = tf.train.AdamOptimizer().minimize(loss_tensor)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
saver = tf.train.Saver()

In [ ]:
num_epoch = 5000

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    train_dict = {
        input_placeholder: x_train,
        target_placeholder: y_train
    }

    test_dict = {
        input_placeholder: x_test,
        target_placeholder: y_test
    }

    best_loss = float('inf')

    for epoch in range(num_epoch):
        sess.run(optimizer, feed_dict=train_dict)

        # MSE
        loss = sess.run(loss_tensor, feed_dict=train_dict)
        val_loss = sess.run(loss_tensor, feed_dict=test_dict)

        # RMSE
        rmse = sess.run(rmse_tensor, feed_dict=train_dict)
        val_rmse = sess.run(rmse_tensor, feed_dict=test_dict)

        if val_loss < best_loss:
            best_loss = val_loss
            saver.save(sess, '/.best_model.ckpt')
        if epoch % 100 == 0:
            print(f'Epoch {epoch + 1} | Loss {loss:.4f} | Val_loss {val_loss:.4f} | RMSE {rmse:.4f} | val_rmse {val_rmse:.4f}')

Epoch 1 | Loss 0.1495 | Val_loss 9.0876 | RMSE 0.3604 | val_rmse 2.9382
Epoch 101 | Loss 0.0008 | Val_loss 0.0529 | RMSE 0.0198 | val_rmse 0.1933
Epoch 201 | Loss 0.0007 | Val_loss 0.0456 | RMSE 0.0180 | val_rmse 0.1781
Epoch 301 | Loss 0.0007 | Val_loss 0.0390 | RMSE 0.0174 | val_rmse 0.1627
Epoch 401 | Loss 0.0007 | Val_loss 0.0342 | RMSE 0.0169 | val_rmse 0.1500
Epoch 501 | Loss 0.0006 | Val_loss 0.0309 | RMSE 0.0164 | val_rmse 0.1407
Epoch 601 | Loss 0.0006 | Val_loss 0.0292 | RMSE 0.0160 | val_rmse 0.1356
Epoch 701 | Loss 0.0006 | Val_loss 0.0280 | RMSE 0.0157 | val_rmse 0.1327
Epoch 801 | Loss 0.0006 | Val_loss 0.0275 | RMSE 0.0153 | val_rmse 0.1313
Epoch 901 | Loss 0.0006 | Val_loss 0.0272 | RMSE 0.0150 | val_rmse 0.1295
Epoch 1001 | Loss 0.0005 | Val_loss 0.0269 | RMSE 0.0147 | val_rmse 0.1279
Epoch 1101 | Loss 0.0005 | Val_loss 0.0271 | RMSE 0.0146 | val_rmse 0.1280
Epoch 1201 | Loss 0.0005 | Val_loss 0.0276 | RMSE 0.0144 | val_rmse 0.1288
Epoch 1301 | Loss 0.0005 | Val_loss 0

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, './best_model.ckpt')

    pred = sess.run(out_tensor, feed_dict ={input_placeholder:x})
    pred = scaler.inverse_transform(pred)

    print(f'Tomorrow BTC: {pred} USD')

ValueError: ignored